In [2]:
import pandas as pd
from Testmodule import 資產負債表, 綜合損益表

user_year = int(input('請輸入西元年份'))
user_quarter = int(input('請輸入上個季度'))

data1 = 綜合損益表.綜合損益表by季(user_year, user_quarter)
data2 = 資產負債表.資產負債表by季(user_year, user_quarter)

resultdf=data1.merge(data2,on='Key',how='inner')

project = {}
project['公司代號'] = resultdf['公司 代號_x']
project['公司名稱'] = resultdf['公司名稱_x']
project['年度'] = resultdf['年度_x']
project['季別'] = resultdf['季別_x']
project['基本每股盈餘_EPS'] = resultdf['基本每股盈餘（元）'] #EPS
project['營業收入'] = resultdf['營業收入']
project['權益總計'] = resultdf['權益總計'] #股東權益
project['資產總計'] = resultdf['資產總計'] #資產總額

project = pd.DataFrame(project)

#test

/Users/edward/Library/CloudStorage/OneDrive-Personal/GitHub/ccClub/Project/Testmodule/綜合損益表.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(res.text)[3]
/Users/edward/Library/CloudStorage/OneDrive-Personal/GitHub/ccClub/Project/Testmodule/綜合損益表.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(res.text)[3]
/Users/edward/Library/CloudStorage/OneDrive-Personal/GitHub/ccClub/Project/Testmodule/綜合損益表.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(res.text)[3]
/Users/edward/Library/CloudStorage/OneDrive-Personal/GitHub/ccClub/Project/Testmodule/綜合損益表.py:11